In [1]:
import eikon as ek
ek.set_app_key('9a430e13c8a944e9894743c7bd608b7e41a432be')

In [2]:
import eikon as ek  # the Eikon Python wrapper package
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
from sklearn.svm import SVC  # sckikit-learn
from sklearn.model_selection import train_test_split
import configparser as cp
from collections import Counter
from datetime import datetime
import statistics
import datetime as dt

# Recommendations of S&P1500 

In [3]:
df,er = ek.get_data(['0#.SPX','0#.SPMIDSM'],['TR.NumOfStrongBuy','TR.NumOfBuy','TR.NumOfHold','TR.NumOfSell','TR.NumOfStrongSell',
                              'TR.NumOfNoOpinion'],
                    {'frq': 'FY', 'SDate':'2020-01-01', 'EDate':'2020-12-31'})
df

,Instrument,Recommendation - Number Of Strong Buy,Recommendation - Number Of Buy,Recommendation - Number Of Hold,Recommendation - Number Of Sell,Recommendation - Number Of Strong Sell,Recommendation - Number Of No Opinion
0,POOL.OQ,2,0,7,0,0,0
1,CHRW.OQ,2,4,12,4,0,0
2,AJG.N,3,4,6,0,0,0
3,CNP.N,2,6,8,0,0,0
4,AMCR.N,1,2,4,1,0,0
...,...,...,...,...,...,...,...
1503,ARNC.N,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1504,WAFD.OQ,0,0,3,0,0,0
1505,THRM.OQ,1,2,5,0,0,0
1506,TMST.N,0,2,3,2,0,0


In [4]:
sumstrongbuy =df['Recommendation - Number Of Strong Buy'].sum(skipna=bool)
sumbuy =df['Recommendation - Number Of Buy'].sum(skipna=bool)
sumhold =df['Recommendation - Number Of Hold'].sum(skipna=bool)
sumsell =df['Recommendation - Number Of Strong Sell'].sum(skipna=bool)
sumstrongsell =df['Recommendation - Number Of Strong Sell'].sum(skipna=bool)
sumnone =df['Recommendation - Number Of No Opinion'].sum(skipna=bool)

In [5]:
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

data = {'Strong Buy': [] , 'Buy' : [],'Hold': [],'Sell':[],'Strong Sell':[],
        'No Opinion':[],}
table = pd.DataFrame(data)

for year in years:
    df,er = ek.get_data(['0#.SPX','0#.SPMIDSM'],['TR.NumOfStrongBuy','TR.NumOfBuy','TR.NumOfHold','TR.NumOfSell','TR.NumOfStrongSell',
                        'TR.NumOfNoOpinion'],
                        {'frq': 'FY', 'SDate':'{}-01-01'.format(year), 'EDate':'{}-12-31'.format(year)})
    sumstrongbuy =df['Recommendation - Number Of Strong Buy'].sum(skipna=bool)
    sumbuy =df['Recommendation - Number Of Buy'].sum(skipna=bool)
    sumhold =df['Recommendation - Number Of Hold'].sum(skipna=bool)
    sumsell =df['Recommendation - Number Of Strong Sell'].sum(skipna=bool)
    sumstrongsell =df['Recommendation - Number Of Strong Sell'].sum(skipna=bool)
    sumnone =df['Recommendation - Number Of No Opinion'].sum(skipna=bool)
    
    data = {'Strong Buy': [sumstrongbuy] , 'Buy' : [sumbuy],'Hold': [sumhold],'Sell':[sumsell],
            'Strong Sell':[sumstrongsell],'No Opinion':[sumnone],}
    table2 = pd.DataFrame(data, index=[year])
    table = pd.concat([table, table2])

table


,Strong Buy,Buy,Hold,Sell,Strong Sell,No Opinion
2010,3437.0,3290.0,5896.0,229.0,229.0,1.0
2011,3741.0,3907.0,6923.0,190.0,190.0,2.0
2012,3715.0,4422.0,6957.0,136.0,136.0,1.0
2013,3538.0,4504.0,7451.0,130.0,130.0,1.0
2014,3709.0,4946.0,7843.0,163.0,163.0,5.0
2015,3794.0,5227.0,7831.0,128.0,128.0,6.0
2016,4011.0,5414.0,7953.0,141.0,141.0,5.0
2017,3699.0,5130.0,8377.0,211.0,211.0,6.0
2018,3860.0,5254.0,7934.0,196.0,196.0,6.0
2019,3975.0,5485.0,7759.0,193.0,193.0,5.0


# Identify 1 stock rating recommendation: POOL.OQ

In [6]:
df1,er1 = ek.get_data(['POOL.OQ'],['TR.AnalystCode','TR.RecMean','TR.NumOfStrongBuy','TR.NumOfBuy','TR.NumOfHold',
                                   'TR.NumOfSell','TR.NumOfStrongSell'],
                    {'frq': 'FY', 'SDate':'2015-01-01', 'EDate':'2015-12-31'})    
analysts=df1['Instrument'].groupby(df1['Analyst - Analyst Code']).count()

analysts

Analyst - Analyst Code
1RER    1
2SOA    1
35VW    1
3UE1    1
4EH7    1
4WV2    1
5Y1B    1
7NMO    1
7W0X    1
Name: Instrument, dtype: int64

In [7]:
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

data1 = {'Strong Buy': [] , 'Buy' : [],'Hold': [],'Sell':[],'Strong Sell':[], 'Total':[]}
table1 = pd.DataFrame(data1)

for year in years:
    df1,er = ek.get_data(['POOL.OQ'],['TR.NumOfStrongBuy','TR.NumOfBuy','TR.NumOfHold','TR.NumOfSell','TR.NumOfStrongSell',
                                     'TR.NumOfRecommendations'],
                        {'frq': 'FY', 'SDate':'{}-01-01'.format(year), 'EDate':'{}-12-31'.format(year)})
    sumstrongbuy =df1['Recommendation - Number Of Strong Buy'].sum(skipna=bool)
    sumbuy =df1['Recommendation - Number Of Buy'].sum(skipna=bool)
    sumhold =df1['Recommendation - Number Of Hold'].sum(skipna=bool)
    sumsell =df1['Recommendation - Number Of Strong Sell'].sum(skipna=bool)
    sumstrongsell =df1['Recommendation - Number Of Strong Sell'].sum(skipna=bool)
    total = df1['Recommendation - Number Of Total'].sum(skipna=bool)
    
    data1 = {'Strong Buy': [sumstrongbuy] , 'Buy' : [sumbuy],'Hold': [sumhold],'Sell':[sumsell],
            'Strong Sell':[sumstrongsell], 'Total':[total]}
    table2 = pd.DataFrame(data1, index=[year])
    table1 = pd.concat([table1, table2])

table1

,Strong Buy,Buy,Hold,Sell,Strong Sell,Total
2010,3.0,0.0,6.0,1.0,1.0,11.0
2011,3.0,0.0,6.0,1.0,1.0,10.0
2012,2.0,1.0,6.0,0.0,0.0,9.0
2013,0.0,3.0,6.0,0.0,0.0,9.0
2014,1.0,3.0,5.0,0.0,0.0,9.0
2015,3.0,1.0,5.0,0.0,0.0,9.0
2016,2.0,2.0,4.0,0.0,0.0,8.0
2017,2.0,2.0,4.0,0.0,0.0,8.0
2018,3.0,0.0,3.0,0.0,0.0,6.0
2019,2.0,0.0,6.0,0.0,0.0,8.0


# Create POOL.OQ Transition Matrix for Reference Point

In [8]:
datafr,err = ek.get_data(['POOL.OQ'],
                         ['TR.RecEstValue',
                        #'TR.RecEstValue.calcdate',
                          'TR.RecEstValue.brokername',
                          'TR.RecEstValue.analystcode'],
                        {'frq': 'FY', 'SDate':'2010-01-01', 'EDate':'2020-12-31'})

datafr.head() 

,Instrument,Standard Rec (1-5) - Broker Estimate,Broker Name,Analyst Code
0,POOL.OQ,3,LONGBOW RESEARCH,4909
1,POOL.OQ,1,"MORGAN KEEGAN & COMPANY, INC.",FK6
2,POOL.OQ,3,HILLIARD LYONS,45L4
3,POOL.OQ,4,PIPER SANDLER COMPANIES,4LF5
4,POOL.OQ,1,Permission Denied 104848,


In [9]:
datafr =datafr[datafr["Broker Name"].str.contains("Permission Denied")==False]
sorteddatafr = datafr.sort_values(by=['Analyst Code','Instrument'])
sorteddatafr = sorteddatafr.dropna(subset=["Standard Rec (1-5) - Broker Estimate", 'Analyst Code'])
sorteddatafr.head()
#This step is for removing "Permission Denied" recommendations where the analyst code is not visible and sorting.

,Instrument,Standard Rec (1-5) - Broker Estimate,Broker Name,Analyst Code
8,POOL.OQ,5,WEDBUSH SECURITIES INC.,1708
20,POOL.OQ,5,WEDBUSH SECURITIES INC.,1708
7,POOL.OQ,1,WILLIAM BLAIR & COMPANY,2GS8
19,POOL.OQ,1,WILLIAM BLAIR & COMPANY,2GS8
26,POOL.OQ,3,TRUIST SECURITIES,2M8D


In [10]:
labels = sorted(sorteddatafr['Standard Rec (1-5) - Broker Estimate'].unique())
labels

[1, 2, 3, 4, 5]

In [11]:
codes = sorted(sorteddatafr['Analyst Code'].unique())
codes

['1708',
 '2GS8',
 '2M8D',
 '35VW',
 '3JLZ',
 '3JYN',
 '45L4',
 '4909',
 '4EUZ',
 '4LF5',
 '4WV2',
 '4XTM',
 '5Y1B',
 '6EB2',
 '70D9',
 '7LSG',
 '7M5P',
 '7NMO',
 '7W0X',
 'FK6',
 'SDA']

In [12]:
matrix = pd.DataFrame(0, index = labels, columns = labels)

for code in codes:
    auxdf = sorteddatafr[sorteddatafr['Analyst Code'] == code]   #creating an auxilliary dataframe for each analyst

    auxdf['next'] = auxdf['Standard Rec (1-5) - Broker Estimate'].shift(-1)

    auxgroups = auxdf.groupby(['Standard Rec (1-5) - Broker Estimate', 'next'])
    auxcounts = {i[0]:len(i[1]) for i in auxgroups}

    x = list(auxcounts.keys())
    y = list(auxcounts.values())
    for i in range(len(x)):
        a = x[i][0]
        b = x[i][1]

    
        y1 = y[i]

        matrix[b][a] += y1    #it does columns first so b before a

matrix

,1,2,3,4,5
1,9,0,1,0,0
2,0,3,2,0,0
3,1,1,24,0,0
4,0,0,1,0,0
5,0,0,0,0,1


# S&P 1500 Data Cleaning

In [13]:
recc,err = ek.get_data(['0#.SPX','0#.SPMIDSM'],
                         ['TR.RecEstValue','TR.RecEstValue.brokername',
                          'TR.RecEstValue.analystcode'],
                      {'frq': 'FY', 'SDate':'2010-01-01', 'EDate':'2020-12-31'}) 
recc

,Instrument,Standard Rec (1-5) - Broker Estimate,Broker Name,Analyst Code
0,POOL.OQ,3,LONGBOW RESEARCH,4909
1,POOL.OQ,1,"MORGAN KEEGAN & COMPANY, INC.",FK6
2,POOL.OQ,3,HILLIARD LYONS,45L4
3,POOL.OQ,4,PIPER SANDLER COMPANIES,4LF5
4,POOL.OQ,1,Permission Denied 104848,
...,...,...,...,...
188507,ACA.N,2,GABELLI & COMPANY,6TF7
188508,ACA.N,2,SUSQUEHANNA FINANCIAL GROUP LLLP,<NA>
188509,ACA.N,1,"OPPENHEIMER & CO., INC.",3GVL
188510,ACA.N,1,Permission Denied 75992,


In [14]:
recc.isnull().sum() #check how many na values in each column to specify in the paper the omitted variables

Instrument                                 0
Standard Rec (1-5) - Broker Estimate    5349
Broker Name                                0
Analyst Code                            2245
dtype: int64

In [15]:
recc =recc[recc["Broker Name"].str.contains("Permission Denied")==False]
recc = recc.dropna(subset=["Standard Rec (1-5) - Broker Estimate", 'Analyst Code'])
recc

,Instrument,Standard Rec (1-5) - Broker Estimate,Broker Name,Analyst Code
0,POOL.OQ,3,LONGBOW RESEARCH,4909
1,POOL.OQ,1,"MORGAN KEEGAN & COMPANY, INC.",FK6
2,POOL.OQ,3,HILLIARD LYONS,45L4
3,POOL.OQ,4,PIPER SANDLER COMPANIES,4LF5
5,POOL.OQ,3,THOMPSON RESEARCH GROUP,4XTM
...,...,...,...,...
188505,ACA.N,2,D.A. DAVIDSON & COMPANY,479S
188506,ACA.N,2,CJS SECURITIES,7SZS
188507,ACA.N,2,GABELLI & COMPANY,6TF7
188509,ACA.N,1,"OPPENHEIMER & CO., INC.",3GVL


In [16]:
labels = sorted(recc['Standard Rec (1-5) - Broker Estimate'].unique())
labels
#observe how there is a 0 rating!!

[0, 1, 2, 3, 4, 5]

# Dropping the zero ratings to remove data error

In [17]:
#checking which rows have a zero value for the broker estimate and then removing them
np.where(recc['Standard Rec (1-5) - Broker Estimate'] == 0)

(array([19207, 19229, 19246, 34677, 34694, 34709, 34722, 34744, 36424,
        36436, 36447, 36456, 36464, 36476, 37735, 37747, 37809, 72659,
        72662, 72670, 72678, 74089, 74096, 74105, 74109, 74115, 74121,
        75366, 85528, 85544, 85565, 85570, 85573, 85581, 85587, 85589],
       dtype=int64),)

In [18]:
#Explanation: I have decided to remove the zero ratings, since it does not correspond to anything according to the formula
#defined by Refinitiv. There are only 36 cases where the rating has been identified as zero out of 188624 cases in our 
#database. Thus, it is a neglectible percent. 

In [19]:
recc = recc[recc['Standard Rec (1-5) - Broker Estimate'] != 0]
labels = sorted(recc['Standard Rec (1-5) - Broker Estimate'].unique())
labels

[1, 2, 3, 4, 5]

In [20]:
sortedrecc = recc.sort_values(by=['Instrument','Analyst Code'])
sortedrecc

,Instrument,Standard Rec (1-5) - Broker Estimate,Broker Name,Analyst Code
62987,A.N,2,CREDIT SUISSE,3JM0
63002,A.N,2,CREDIT SUISSE,3JM0
62999,A.N,2,COWEN AND COMPANY,3QGX
63021,A.N,2,COWEN AND COMPANY,3QGX
63024,A.N,2,COWEN AND COMPANY,3QGX
...,...,...,...,...
153734,ZYXI.OQ,1,LADENBURG THALMANN & CO. INC.,690U
153738,ZYXI.OQ,1,LADENBURG THALMANN & CO. INC.,690U
153739,ZYXI.OQ,2,"H.C. WAINWRIGHT & CO., LLC",72QG
153736,ZYXI.OQ,2,B. RILEY SECURITIES,7HTH


In [21]:
stocks = sorted(sortedrecc['Instrument'].unique())
codes = sorted(sortedrecc['Analyst Code'].unique())

# Table II total number of rating changes

In [22]:
matrix = pd.DataFrame(0, index = labels, columns = labels)
ecem = 1
for stock in stocks:
    subdf = sortedrecc[sortedrecc['Instrument'] == stock]
    for code in codes:
        auxdf = subdf[subdf['Analyst Code'] == code]   #creating an auxilliary dataframe for each analyst

        auxdf['next'] = auxdf['Standard Rec (1-5) - Broker Estimate'].shift(-1)

        auxgroups = auxdf.groupby(['Standard Rec (1-5) - Broker Estimate', 'next'])
        auxcounts = {i[0]:len(i[1]) for i in auxgroups}

        x = list(auxcounts.keys())
        y = list(auxcounts.values())
        for i in range(len(x)):
            a = x[i][0]
            b = x[i][1]
            y1 = y[i]
    
            matrix[b][a] += y1   #it does columns first so b before a
    ecem+=1
    
matrix

,1,2,3,4,5
1,14840,771,3006,35,55
2,310,17122,3585,147,8
3,2566,3390,24395,687,294
4,14,164,601,954,11
5,51,10,278,29,382


# Median Days Between Changes

In [23]:
days,err = ek.get_data(['0#.SPX','0#.SPMIDSM'],
                         ['TR.RecEstValue','TR.RecEstValue.analystcode','TR.RecEstValue.brokername',
                         'TR.RecEstValue.date'],
                      {'frq': 'FY', 'SDate':'2010-01-01', 'EDate':'2020-12-31'}) 
days

,Instrument,Standard Rec (1-5) - Broker Estimate,Analyst Code,Broker Name,Date
0,POOL.OQ,3,4909,LONGBOW RESEARCH,2009-04-03T00:00:00Z
1,POOL.OQ,1,FK6,"MORGAN KEEGAN & COMPANY, INC.",2009-02-20T00:00:00Z
2,POOL.OQ,3,45L4,HILLIARD LYONS,2009-02-23T00:00:00Z
3,POOL.OQ,4,4LF5,PIPER SANDLER COMPANIES,2010-01-05T00:00:00Z
4,POOL.OQ,1,,Permission Denied 104848,2010-02-09T00:00:00Z
...,...,...,...,...,...
188507,ACA.N,2,6TF7,GABELLI & COMPANY,2018-11-19T00:00:00Z
188508,ACA.N,2,<NA>,SUSQUEHANNA FINANCIAL GROUP LLLP,2018-11-14T00:00:00Z
188509,ACA.N,1,3GVL,"OPPENHEIMER & CO., INC.",2019-03-04T00:00:00Z
188510,ACA.N,1,,Permission Denied 75992,2019-09-18T00:00:00Z


In [42]:
days['Broker Name'].unique()

<StringArray>
[             'LONGBOW RESEARCH', 'MORGAN KEEGAN & COMPANY, INC.',
                'HILLIARD LYONS',       'PIPER SANDLER COMPANIES',
       'THOMPSON RESEARCH GROUP',       'WILLIAM BLAIR & COMPANY',
       'WEDBUSH SECURITIES INC.',             'TRUIST SECURITIES',
         'WUNDERLICH SECURITIES',  'KEYBANC CAPITAL MARKETS INC.',
 ...
                    'M PARTNERS',   'SMALL CAP CONSUMER RESEARCH',
 'SOLEIL - POMEROY RESEARCH LLC',      'CANTOR FITZGERALD CANADA',
 'DISCOVERY INVESTMENT RESEARCH',             'MDB CAPITAL GROUP',
       'EMPIRE ASSET MANAGEMENT',                     'HELVEA SA',
      'ALERE FINANCIAL PARTNERS',    'BRINSON PATRICK SECURITIES']
Length: 373, dtype: string

In [43]:
days['Analyst Code'].unique()

<StringArray>
['4909',  'FK6', '45L4', '4LF5', '4XTM', '2GS8', '1708',  'SDA', '4EUZ',
 '2M8D',
 ...
 '4AHK', '6OT4', '6WXM', '78VU', '6BBS', '3S7E', '6MBQ', '4GWW', '6RIL',
 '4GSP']
Length: 5011, dtype: string

In [24]:
days =days[days["Broker Name"].str.contains("Permission Denied")==False]
days = days.dropna(subset=['Standard Rec (1-5) - Broker Estimate','Analyst Code'])

In [25]:
days.isnull().sum()

Instrument                              0
Standard Rec (1-5) - Broker Estimate    0
Analyst Code                            0
Broker Name                             0
Date                                    0
dtype: int64

In [26]:
days['Date'] = pd.to_datetime(days['Date']).dt.date

In [27]:
start_date = pd.to_datetime("2010-1-1").date()
end_date = pd.to_datetime("2020-12-31").date()
after_start_date = days['Date'] >= start_date
before_end_date = days['Date'] <= end_date
between_two_dates = after_start_date & before_end_date
filtered = days.loc[between_two_dates]

In [28]:
filtered=filtered.drop_duplicates()
filtered['Date']=pd.to_datetime(filtered['Date'],errors='coerce')
filtered['Date']=filtered['Date'].dt.floor('d')

In [29]:
filtered[filtered['Instrument']=='IBM.N'].head(20)

,Instrument,Standard Rec (1-5) - Broker Estimate,Analyst Code,Broker Name,Date
33381,IBM.N,1,1REB,"WELLS FARGO SECURITIES, LLC",2010-08-11
33382,IBM.N,3,4LQ5,"MORNINGSTAR, INC.",2010-10-21
33383,IBM.N,2,4M8X,COLLINS STEWART LLC,2010-11-05
33387,IBM.N,1,4SSI,"STIFEL NICOLAUS AND COMPANY, INCORPORATED",2010-07-13
33403,IBM.N,3,1REB,"WELLS FARGO SECURITIES, LLC",2011-07-07
33404,IBM.N,2,4IQT,MACQUARIE RESEARCH,2011-10-14
33406,IBM.N,3,64KI,FBN SECURITIES,2011-10-18
33409,IBM.N,3,4LQ5,"MORNINGSTAR, INC.",2011-11-08
33410,IBM.N,3,3V6D,HAMBURGER SPARKASSE (HASPA),2011-12-08
33413,IBM.N,2,69AP,EVERCORE ISI,2011-11-03


In [30]:
labels1 = sorted(filtered['Standard Rec (1-5) - Broker Estimate'].unique())
labels1

[0, 1, 2, 3, 4, 5]

In [31]:
filtered = filtered[filtered['Standard Rec (1-5) - Broker Estimate'] != 0]
labels = sorted(filtered['Standard Rec (1-5) - Broker Estimate'].unique())
labels

[1, 2, 3, 4, 5]

In [32]:
sorteddays = filtered.sort_values(by=['Instrument','Analyst Code'])
sorteddays

,Instrument,Standard Rec (1-5) - Broker Estimate,Analyst Code,Broker Name,Date
62999,A.N,2,3QGX,COWEN AND COMPANY,2011-07-25
63024,A.N,2,3QGX,COWEN AND COMPANY,2013-06-13
63058,A.N,3,3QGX,COWEN AND COMPANY,2014-11-18
63071,A.N,2,3QGX,COWEN AND COMPANY,2016-01-07
63087,A.N,3,3QGX,COWEN AND COMPANY,2017-01-04
...,...,...,...,...,...
153729,ZYXI.OQ,2,4SJA,ZACKS SMALL-CAP RESEARCH,2011-02-14
153731,ZYXI.OQ,3,4SJA,ZACKS SMALL-CAP RESEARCH,2013-08-11
153734,ZYXI.OQ,1,690U,LADENBURG THALMANN & CO. INC.,2018-06-28
153739,ZYXI.OQ,2,72QG,"H.C. WAINWRIGHT & CO., LLC",2019-09-23


In [33]:
sorteddays['diff'] = sorteddays.groupby(['Analyst Code','Instrument'])['Date'].diff() / np.timedelta64(1, 'D')
sorteddays['diff'] = sorteddays['diff'].fillna(0)
print(len(sorteddays))
sorteddays[sorteddays['Instrument'] =='IBM.N'].head(10)

46658


,Instrument,Standard Rec (1-5) - Broker Estimate,Analyst Code,Broker Name,Date,diff
33451,IBM.N,3,1BMC,COWEN AND COMPANY,2013-12-06,0.0
33454,IBM.N,3,1BV9,ARGUS RESEARCH CORPORATION,2013-12-06,0.0
33500,IBM.N,1,1BV9,ARGUS RESEARCH CORPORATION,2015-08-26,628.0
33580,IBM.N,3,1BV9,ARGUS RESEARCH CORPORATION,2018-10-18,1149.0
33574,IBM.N,3,1ED4,"WELLS FARGO SECURITIES, LLC",2018-07-30,0.0
33599,IBM.N,3,1ED4,"WELLS FARGO SECURITIES, LLC",2019-12-11,499.0
33381,IBM.N,1,1REB,"WELLS FARGO SECURITIES, LLC",2010-08-11,0.0
33403,IBM.N,3,1REB,"WELLS FARGO SECURITIES, LLC",2011-07-07,330.0
33414,IBM.N,3,2WHJ,"DAVENPORT & COMPANY, LLC",2011-07-06,0.0
33448,IBM.N,4,3E4E,CREDIT SUISSE,2013-08-06,0.0


## Trying with one analyst-stock pairing

In [34]:
n=5
lists = [[[],[],[],[],[]] for i in range (n)]

subdataframe = sorteddays[sorteddays['Instrument'] == 'A.N']
auxdata = subdataframe[subdataframe['Analyst Code'] == '3JM0']

print(auxdata)

for i in range(1,len(auxdata)):
#    print(int(auxdata[i-1:i]['Standard Rec (1-5) - Broker Estimate'])-1)
#    print(int(auxdata[i:i+1]['Standard Rec (1-5) - Broker Estimate'])-1),
#    print(int(auxdata[i:i+1]['diff']))
    
    lists[int(auxdata[i-1:i]['Standard Rec (1-5) - Broker Estimate'])-1][int(auxdata[i:i+1]['Standard Rec (1-5) - Broker Estimate'])-1].append(int(auxdata[i:i+1]['diff']))

for i in range(len(lists)):
    for j in range(len(lists)):
        if lists[i][j] == []:
            continue
        else:
            lists[i][j] = int(statistics.median(lists[i][j]))
lists

Empty DataFrame
Columns: [Instrument, Standard Rec (1-5) - Broker Estimate, Analyst Code, Broker Name, Date, diff]
Index: []


[[[], [], [], [], []],
 [[], [], [], [], []],
 [[], [], [], [], []],
 [[], [], [], [], []],
 [[], [], [], [], []]]

## Running the loop for all stocks

In [35]:
n = 5
lists = [[[],[],[],[],[]] for i in range (n)]

stocks = sorted(sorteddays['Instrument'].unique())

ecem = 1
for stock in stocks:
    subdataframe = sorteddays[sorteddays['Instrument'] == stock]
    codes = sorted(subdataframe['Analyst Code'].unique())
    for code in codes: ##try with another for loop for the stocks
        auxdf = subdataframe[subdataframe['Analyst Code'] == code]
        
        if len(auxdf) == 1:
            continue
        if len(auxdf) == 2:
            lists[int(auxdf[:1]['Standard Rec (1-5) - Broker Estimate'])-1][int(auxdf[1:2]['Standard Rec (1-5) - Broker Estimate'])-1].append(int(auxdf[1:2]['diff']))
        else:
            for i in range(1,len(auxdf)):
                    lists[int(auxdf[i-1:i]['Standard Rec (1-5) - Broker Estimate'])-1][int(auxdf[i:i+1]['Standard Rec (1-5) - Broker Estimate'])-1].append(int(auxdf[i:i+1]['diff']))

    ecem += 1
                    
for i in range(len(lists)):
    for j in range(len(lists)):
        if lists[i][j] == []:
            continue
        else:
            lists[i][j] = int(statistics.median(lists[i][j]))      
lists

[[568, 597, 548, 523, 575],
 [452, 562, 503, 478, 350],
 [495, 448, 556, 537, 521],
 [405, 378, 346, 452, 309],
 [405, 391, 370, 341, 447]]

# Table II Median Days

In [36]:
matrix2 = pd.DataFrame(lists, index = labels, columns = labels)
matrix2

,1,2,3,4,5
1,568,597,548,523,575
2,452,562,503,478,350
3,495,448,556,537,521
4,405,378,346,452,309
5,405,391,370,341,447


# Checking whether NA values could mean dropping a stock

In [37]:
recc1,err = ek.get_data(['0#.SPX','0#.SPMIDSM'],
                         ['TR.RecEstValue','TR.RecEstValue.calcdate','TR.RecEstValue.brokername',
                          'TR.RecEstValue.analystcode'],
                      {'frq': 'FY', 'SDate':'2010-01-01', 'EDate':'2020-12-31'}) 
recc1[recc1['Standard Rec (1-5) - Broker Estimate'].isnull()].head()

#it seems to be that NA value is not dropped stocks, but rather specific to certain brokers that do not provide...
#...broker estimate value access
#watch out for: 1)UBS Equities and 2)Vertical Research Partners

,Instrument,Standard Rec (1-5) - Broker Estimate,Calc Date,Broker Name,Analyst Code
112,CHRW.OQ,<NA>,2010-02-02,UBS EQUITIES,2S8B
119,CHRW.OQ,<NA>,2011-02-01,UBS EQUITIES,2S8B
163,CHRW.OQ,<NA>,2012-01-31,UBS EQUITIES,2S8B
180,CHRW.OQ,<NA>,2013-02-05,UBS EQUITIES,2S8B
275,CHRW.OQ,<NA>,2016-02-02,UBS EQUITIES,70DT


# Checking whether Na values mean dropping a stock with CHRW.OQ-UBS Equities Pairing

In [38]:
recc2,err = ek.get_data(['CHRW.OQ'],
                         ['TR.RecEstValue','TR.RecEstValue.calcdate','TR.RecEstValue.brokername',
                          'TR.RecEstValue.analystcode'],
                      {'frq': 'FY', 'SDate':'2010-01-01', 'EDate':'2020-12-31'}) 

In [39]:
recc2 =recc2[recc2["Broker Name"].str.contains("Permission Denied")==False]
recc2.head()

,Instrument,Standard Rec (1-5) - Broker Estimate,Calc Date,Broker Name,Analyst Code
0,CHRW.OQ,3,2010-02-02,"AVONDALE PARTNERS, LLC",4713
1,CHRW.OQ,1,2010-02-02,"THOMPSON, DAVIS & COMPANY",6COT
4,CHRW.OQ,3,2010-02-02,PIPER SANDLER COMPANIES,4GJT
5,CHRW.OQ,1,2010-02-02,"WELLS FARGO SECURITIES, LLC",4IST
6,CHRW.OQ,1,2010-02-02,CFRA EQUITY RESEARCH,5SZH


In [40]:
recc2.loc[recc2['Broker Name'] == 'UBS EQUITIES']
#therefore: UBS equities does not provide broker estimate publicly at all!!

,Instrument,Standard Rec (1-5) - Broker Estimate,Calc Date,Broker Name,Analyst Code
16,CHRW.OQ,<NA>,2010-02-02,UBS EQUITIES,2S8B
23,CHRW.OQ,<NA>,2011-02-01,UBS EQUITIES,2S8B
67,CHRW.OQ,<NA>,2012-01-31,UBS EQUITIES,2S8B
84,CHRW.OQ,<NA>,2013-02-05,UBS EQUITIES,2S8B
179,CHRW.OQ,<NA>,2016-02-02,UBS EQUITIES,70DT
207,CHRW.OQ,<NA>,2017-01-31,UBS EQUITIES,70DT
221,CHRW.OQ,<NA>,2018-01-30,UBS EQUITIES,70DT
255,CHRW.OQ,<NA>,2019-01-29,UBS EQUITIES,70DT
266,CHRW.OQ,<NA>,2020-01-28,UBS EQUITIES,70DT
